# general project

## Why use the laplace transform 
We use the Laplace transform because it turns a complicated system of differential equations into algebra, making it possible to solve a coupled mechanical system with initial conditions.
## why do we need to utilize the inverse laplace transform
Now the laplace basically describes the motion of two masses.
> X <sub> 1 </sub> and X <sub> 2 <sub>

However, this isnt over time, instead they are described in **laplace domain**. 



Due to this "caviat" there is multiple things we could not evaluate.
** What we CANT see (and/or model) **
1. displacement over time
2. Oscilations (since they happen over time)
3. Both masses moving in tandem
4. Overshoots
5. When both masses settle 
6. Propagation of natural conditions

Therefore our current task should be in essence to find the inverse laplace to evaluate the function over time.

### Use the function from the laplace modeling and then use the inverse
> $$
 X_1(s) =  \frac{ (0.0001677s^3-0.006476s^2+0.8569s+0.3328) }{ (0.4441s^4 + 1.8689s^3 + 89.5745s^2 + 147.429s + 3039.57)}
 $$

> $$
    X_2(s)= \frac{(0.02392s^3 - 0.1239s^2 + 147.429s +2.3942)} {(0.4441s^4 + 1.8689s^3 + 89.5745s^2 + 147.429s + 3039.57)}
    $$

Useing these two laplace transforms we can start trying to find the inverse laplace transform to write the models of the weights based on time.

In [1]:
# first lets try and get the roots of the denominator we do this via a method from the libery numpy
import numpy as np

#use the coeficients of the denominator
# 0.4441*s^4 + 1.8689*s^3 + 89.5745*s^2 + 147.429*s + 3039.57
coeffs = [0.4441, 1.8689, 89.5745, 147.429, 3039.57]

#get the roots using numpy 
roots = np.roots(coeffs)

print("The roots of the denominator are:")
for r in roots:
    print(r)

The roots of the denominator are:
(-1.4408641620027467+12.29326569071687j)
(-1.4408641620027467-12.29326569071687j)
(-0.6632790489857681+6.65099629098637j)
(-0.6632790489857681-6.65099629098637j)


In this case we get $$ −1.44086416±12.29326569j; -0.6632790489857681±6.65099629098637j $$ as our current roots (NOTE: the j is the equivalent of i so both are imaginaryvalues)

To find the laplace transform of our function we basically used partial fraction expansion:
$$
X(s)=\sum_{4}^{k=1} \frac{A_k} {s-s_k}
$$
In this function we use 
>$$ s_k 
$$

which are the roots of the denominatorand 
>$$ A_k $$

which are residues of the numeratos of the partial fractions which should follow the next procedure 
$$ A_k = Res{X(s), s=s_k}$$
which is why for inverse laplace we use 
$$ x(t)= \sum _{k=1}^{4} A_ke^{t*s_k} $$

Now based on the tecnicallity that our laplace transform is 

$$
X(s)=\frac{N(s)}{D(s)}
$$

and in the case of being near a simple pole \(s_k\).



$$
D(s_k)=0,
$$

which allows us to rewrite the following expression:

$$
D(s) = (s - s_k)\, Q(s)
$$

with

$$
Q(s_k) \neq 0.
$$

Then:

$$
X(s)=\frac{N(s)}{(s - s_k)\, Q(s)}.
$$

Write the local expansion:

$$
X(s)=\frac{A_k}{s - s_k} + \text{regular terms}.
$$

Multiply both sides by \((s - s_k)\):

$$
(s - s_k)\,X(s)=A_k + (s - s_k)\,\text{regular terms}.
$$

Now take the limit as \(s \to s_k\):

$$
A_k = \lim_{s \to s_k} (s - s_k)\, X(s).
$$

Substitute \(X(s)=\frac{N(s)}{D(s)}\):

$$
A_k = \lim_{s \to s_k} 
\frac{(s - s_k)N(s)}{(s - s_k)Q(s)}.
$$

Cancel the \((s - s_k)\):

$$
A_k = \frac{N(s_k)}{Q(s_k)}.
$$

And since

$$
Q(s)=\frac{D(s)}{s - s_k}
\qquad\Rightarrow\qquad
Q(s_k)=D'(s_k),
$$

we obtain the residue formula:

$$
\boxed{
A_k = \frac{N(s_k)}{D'(s_k)}
}
$$

This is the formula used in Python to compute the residues.

In [12]:
import numpy as np
import sympy as sp

# Symbols
s, t = sp.symbols('s t')

# Define numerator and denominator
num1 = 0.0001677*s**3 - 0.006476*s**2 + 0.8569*s + 0.3328
num2 = 0.02392*s**3 - 0.1239*s**2 + 147.429*s + 2.3942
den  = 0.4441*s**4 + 1.8689*s**3 + 89.5745*s**2 + 147.429*s + 3039.57

# 1. Numeric roots using numpy
coeffs = [0.4441, 1.8689, 89.5745, 147.429, 3039.57]
roots_np = np.roots(coeffs)

print("Numeric poles:")
print(roots_np)

# Convert numpy complex to sympy complex using sp.N()
sym_roots = [sp.N(r) for r in roots_np]

# Derivative of denominator
den_prime = sp.diff(den, s)

# 2. Compute residues using N(s_k) / D'(s_k)
residues_X1 = [
    sp.N(num1.subs(s, sym_roots[i]) / den_prime.subs(s, sym_roots[i]))
    for i in range(4)
]

residues_X2 = [
    sp.N(num2.subs(s, sym_roots[i]) / den_prime.subs(s, sym_roots[i]))
    for i in range(4)
]

print("\nResidues for X1(s):")
print(residues_X1)

print("\nResidues for X2(s):")
print(residues_X2)

# 3. Build x(t) = Σ A_k e^(s_k t)
x1_t = sum(residues_X1[i] * sp.exp(sym_roots[i] * t) for i in range(4))
x2_t = sum(residues_X2[i] * sp.exp(sym_roots[i] * t) for i in range(4))

print("\nx1(t):")
print(x1_t)

print("\nx2(t):")
print(x2_t)


Numeric poles:
[-1.44086416+12.29326569j -1.44086416-12.29326569j
 -0.66327905 +6.65099629j -0.66327905 -6.65099629j]

Residues for X1(s):
[-0.00870896824269133 + 0.00171511294576554*I, -0.00870896824269133 - 0.00171511294576554*I, 0.00889777706953211 - 0.000955029212729211*I, 0.00889777706953211 + 0.000955029212729211*I]

Residues for X2(s):
[-1.507302828721 + 0.119463301647024*I, -1.507302828721 - 0.119463301647024*I, 1.53423370014635 - 0.00925757848304954*I, 1.53423370014635 + 0.00925757848304954*I]

x1(t):
(-0.00870896824269133 - 0.00171511294576554*I)*exp(t*(-1.44086416200275 - 12.2932656907169*I)) + (-0.00870896824269133 + 0.00171511294576554*I)*exp(t*(-1.44086416200275 + 12.2932656907169*I)) + (0.00889777706953211 + 0.000955029212729211*I)*exp(t*(-0.663279048985768 - 6.65099629098637*I)) + (0.00889777706953211 - 0.000955029212729211*I)*exp(t*(-0.663279048985768 + 6.65099629098637*I))

x2(t):
(-1.507302828721 - 0.119463301647024*I)*exp(t*(-1.44086416200275 - 12.2932656907169*I)) 

NOTE : I is the same as i from imaginary numbers just thats how it looks in numpy